In [77]:
import pandas as pd
import numpy as np
from collections import defaultdict


In [78]:
df_ped = pd.read_csv('ped_annotations.csv')
removed_vehicles_df_pred = df_ped[df_ped["object-name"].str.contains("vehicle") == False]

removed_vehicles_df_pred.head()

,clip_id,frame_number,timestamp,height,width,top,left,label_category_attr:occlusion,object-id,object-name
0,cqibo8l2ijgtebr1,0,0.000,90.0,27.0,495.0,262.0,not-occluded,9330fbc0-96b5-11eb-9dd2-bf31d9c6663f,pedestrian:1
1,cqibo8l2ijgtebr1,0,0.000,65.0,19.0,515.0,336.0,not-occluded,96e37c70-96b5-11eb-9dd2-bf31d9c6663f,pedestrian:2
2,cqibo8l2ijgtebr1,0,0.000,94.0,19.0,496.0,362.0,major,9b599b40-96b5-11eb-9dd2-bf31d9c6663f,pedestrian:3
3,cqibo8l2ijgtebr1,0,0.000,99.0,26.0,492.0,369.0,not-occluded,a101aa60-96b5-11eb-9dd2-bf31d9c6663f,pedestrian:4
4,cqibo8l2ijgtebr1,30,1.001,90.0,27.0,495.0,215.0,not-occluded,9330fbc0-96b5-11eb-9dd2-bf31d9c6663f,pedestrian:1


In [79]:
df_car = pd.read_csv(f'/home/cvrr/Documents/Samveed/p/final_trajectories.csv')
# df_clip_car = df_car.loc[df_car["clipID"] == clip_id]
df_car.head()

,ts_unix,lat,lon,height,h_accuracy,v_accuracy,n_sats,flags,clipID,ts,date
0,1483232570164,32.880114,-117.235346,74.836757,115,258,10,11,cjrbrdbxm5rgj3pa,2017-01-01 01:02:50.164,2017-01-01
1,1483232570296,32.880114,-117.235346,74.830534,115,258,10,11,cjrbrdbxm5rgj3pa,2017-01-01 01:02:50.296,2017-01-01
2,1483232570413,32.880114,-117.235346,74.825411,115,258,10,11,cjrbrdbxm5rgj3pa,2017-01-01 01:02:50.413,2017-01-01
3,1483232570526,32.880114,-117.235346,74.819530,115,258,10,11,cjrbrdbxm5rgj3pa,2017-01-01 01:02:50.526,2017-01-01
4,1483232570645,32.880114,-117.235346,74.814969,115,257,10,11,cjrbrdbxm5rgj3pa,2017-01-01 01:02:50.645,2017-01-01


In [196]:
ped_car_common_clips = set(df_car["clipID"]).intersection(set(removed_vehicles_df_pred["clip_id"] ))

In [197]:
ped_car_common_clips

{'c007a6dwfphch1eq',
 'c0eyi3xbeszjylx5',
 'c0hgwlfzypjd482y',
 'c0imwqpwewlqor0z',
 'c0rifduuvgejd2mt',
 'c12ec6v5ib3fj9sc',
 'c1gt033903vh422s',
 'c1kqfish1h1yw1se',
 'c1pq5gzob136hbn6',
 'c1r3tdla7d6klva8',
 'c1zp9x729fdydjds',
 'c240s35nnevvmhwa',
 'c2a3d86ers6e3vyz',
 'c2j1oki3cn95dsec',
 'c2oarlx7p88cq1l2',
 'c2rske9fja7yi9cq',
 'c38qs2eppped30ro',
 'c3cv1rekz0jn1ebd',
 'c3f9nx9dquwhvyou',
 'c3h3bk64gcfe41mc',
 'c3i88l33p5fm5m6l',
 'c3qs4owrd8p3f6m4',
 'c3sj3na1duw2hv5d',
 'c3u4xkju16f5ivcc',
 'c3xr617eypf6qmne',
 'c45kyc6oi9wsl5b3',
 'c4612sucz07ppkry',
 'c4jkxqjdffnbtm87',
 'c4pg45yas4nspvw2',
 'c4wrt3o37m3ik25u',
 'c57qk2yog9rkjrof',
 'c58c0ijvhbx8l00m',
 'c5jzq0eg89s7y6o4',
 'c5nil0qsbqpd7fi1',
 'c5pltzrnjd12k113',
 'c5v17d6ngf4lkw7r',
 'c673utpbvqm5se9z',
 'c6dw61urwx7k2ont',
 'c6eidtbptxn8ls4v',
 'c6i7ki02bvg7g0ht',
 'c6rvapb7j1u91wlc',
 'c6z0ck532ftn043o',
 'c6zc48xbxkijvrgt',
 'c7868unpeoikogoz',
 'c7ahpez6ehqonv7d',
 'c7b6bfujn5hzms70',
 'c7cet1578nkvyevv',
 'c7gukmhno0a

In [81]:
print(len(set(removed_vehicles_df_pred["clip_id"])))
print(len(set(df_car["clipID"])))
print(len(ped_car_common_clips))

455
3942
220


In [266]:

ped_count = defaultdict(dict)

# ped_count: nested_dict

# Args:
#     1st key: ped_count["clipID"]: Pass the appropriate clip ID
#     2nd key: ped_count["clipID"]["frame_number"]: Pass the frame number/second info i.e which second of the 10s clip you want to reference
    
#     ped_count["clipID"]["frame_number"] = {num_peds: location_dataframe}
#     Gives the number of pedestrians and the location range of those pedestrians

#     location_dataframe: Cols = [ts_unix", "lat", "lon", "ts"]
x = pd.DataFrame(columns=["lat", "lon"])
ped_location_and_time = pd.DataFrame(columns=["ts_unix", "ped_count", "lat", "lon", "ts"])
for common_clip in list(ped_car_common_clips)[1:]:
    df_clip_ped = removed_vehicles_df_pred.loc[removed_vehicles_df_pred["clip_id"] == common_clip]
    df_clip_car = df_car.loc[df_car["clipID"] == common_clip]

    number_of_lat_long = len(df_clip_car["lat"])
    number_of_coords_per_sec = number_of_lat_long//10

    modified_df = df_clip_car.filter(["ts_unix", "lat", "lon", "ts"], axis = 1)
    lat_long_info = df_clip_car.filter(["lat", "lon"], axis=1)
    coords_per_frame = np.array_split(modified_df, 10)
    x = x.append(lat_long_info)
    for frame in (set(df_clip_ped["frame_number"])):
        index = int(frame/30)
        num_peds = len(set(df_clip_ped.loc[df_clip_ped["frame_number"] == frame]["object-id"]))
        ped_count[common_clip][index] = {num_peds: coords_per_frame[index]}
        ts_unix, lat, lon, ts = coords_per_frame[index].iloc[len(coords_per_frame[index])//2]
        ped_location_and_time = ped_location_and_time.append({"ts_unix": ts_unix, "ped_count": num_peds, "lat": lat, "lon": lon, "ts": ts}, ignore_index=True)

In [103]:
x.to_csv("/home/cvrr/Documents/Samveed/p/lat_long_info.csv", index=False)

In [298]:
# ped_location_and_time --> gives the pedestrian count and time stamp corresponding to all pedestrians we found
all_ped_locations = ped_location_and_time[["lat", "lon"]]

In [276]:
test_df = pd.read_csv('/home/cvrr/Documents/Samveed/p/ins_data/c0eyi3xbeszjylx5/position_rover_llh.txt', header=None)
test_data = test_df.iloc[:, 1:3].values

In [303]:
from math import radians, cos, sin, asin, sqrt, degrees, atan2

# p1 = [  32.86702567346866, -117.250401337591  ]
# p2 = [32.8666369415, -117.2497705260]

def validate_point(p):
    lat, lon = p
    assert -90 <= lat <= 90, "bad latitude"
    assert -180 <= lon <= 180, "bad longitude"

# original formula from  http://www.movable-type.co.uk/scripts/latlong.html
def distance_haversine(p1, p2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    Haversine
    formula: 
        a = sin²(Δφ/2) + cos φ1 ⋅ cos φ2 ⋅ sin²(Δλ/2)
                        _   ____
        c = 2 ⋅ atan2( √a, √(1−a) )
        d = R ⋅ c

    where   φ is latitude, λ is longitude, R is earth’s radius (mean radius = 6,371km);
            note that angles need to be in radians to pass to trig functions!
    """
    lat1, lon1 = p1
    lat2, lon2 = p2
    for p in [p1, p2]:
        validate_point(p)

    R = 6371 # km - earths's radius

    # convert decimal degrees to radians 
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    # haversine formula 
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) # 2 * atan2(sqrt(a), sqrt(1-a))
    d = R * c
    return d*1000

# distance_haversine(p1, p2)/1000

0.07307149803731341

In [292]:
test_data[0].reshape(1, -1).shape

(1, 2)

In [312]:
import numpy as np
from sklearn import metrics
from sklearn.neighbors import BallTree
R = 6371008.7714
a = np.array([32.86702567346866, -117.250401337591])
X = np.radians(a.reshape(1,-1))
b = np.array([32.8666369415, -117.2497705260])
tree = BallTree(X, leaf_size=2, metric='haversine')
# print(test_data[0].reshape(1, -1))              
dist, ind = tree.query(np.radians(b.reshape(1, -1)), k=1)                
print(ind)  # indices of 3 closest neighbors
print(all_ped_locations.iloc[ind[0]])
print(dist*R)  # distances to 3 closest neighbors

[[0]]
            lat             lon
0 32.8768000000 -117.2333000000
[[ 73.07159863994062]]


In [307]:
import numpy as np
from sklearn import metrics
from sklearn.neighbors import BallTree
R = 6371008.7714
X = np.radians(all_ped_locations.values)

tree = BallTree(X, leaf_size=2, metric='haversine')
print(test_data[0].reshape(1, -1))              

def get_nearest_ped_coordinates(curr_pos, k=1):
    
    dist, ind = tree.query(np.radians(curr_pos.reshape(1, -1)), k=k)                
    # print(ind)  # indices of k closest neighbors
    closest_coordinates = (all_ped_locations.iloc[ind[0]].values)
    closest_distances = (dist*R)  # distances to k closest neighbors, in metres

    return closest_coordinates, closest_distances

[[  32.86702567346866 -117.250401337591  ]]
[[773 406 407]]
[[  32.86663694154288 -117.24977052603545]
 [  32.87141097231807 -117.24724380420791]
 [  32.87145800781288 -117.247110113242  ]]
[[  73.07159314993585  569.85941963846142  580.85083542397672]]


In [ ]:
org_pos = test_data[0]
def get_alert(curr_pos, high_vel_dist_alert_th, low_vel_dist_alert_th, car_dist_th=5.0, vel_th=50.0):
    """
    curr_pos = numpy.array --> (lat, lon)
    high_vel_dist_alert_th --> 
    low_vel_dist_alert_th --> 
    car_dist_th = float --> calcualting nearest pedestrian after every 'car_dist_th' metres
    vel_th = float --> alerting based on different velocities, subject to threshold
    """
    curr_dist = distance_haversine(org_pos, curr_pos)

    if curr_dist > car_dist_th:
        
        nearest_ped_locs, nearest_ped_distances = get_nearest_ped_coordinates(curr_pos, 3)
        if vel_car > vel_th:
            # check for alert condition w.r.t high_vel_dist_alert_th
        else:
            # check for alert condition w.r.t low_vel_dist_alert_th


## Using Ball from sklearn library to calculate distance between (lat,lon) pairs

In [188]:
# from sklearn.neighbors import BallTree
# pd.set_option('display.float_format', '{:.10f}'.format)

In [190]:
# import pandas as pd
# pd.set_option('display.float_format', '{:.10f}'.format)
# lat_lon = pd.read_csv("/home/cvrr/Documents/Samveed/p/lat_long_info.csv")
# np.set_printoptions(precision=14)
# ball = BallTree(np.array(np.array(lat_lon[["lat", "lon"]])[:-1]), metric='haversine')
# k = 1
# curr_lat_lon_query = np.array([[  32.84233682983939,-117.27313288622216]])
# distances, indices = ball.query(curr_lat_lon_query, k=k)
# distances = distances*6371
